In [ ]:
import sqlite3, getpass
import pandas as pd
import polars as pl
from itables import show
# import itables.options as opt
# opt.maxBytes = 131072

In [ ]:
sqlite_dir = getpass.getpass(prompt='Base directory for SQLite files : ')

In [ ]:
#connection = sqlite3.connect(f"{sqlite_dir}synop_b.2024031112.sqlite")

connection = sqlite3.connect(f"{sqlite_dir}2024031318__sqlite")
cursor = connection.cursor()

query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(query)
tables = cursor.fetchall() # ('DATA',) ('HEADER',) ('OmF',)

for table in tables:
    print(table)

connection.close()

In [ ]:
# connection = sqlite3.connect(f"{sqlite_dir}synop_b.2024031112.sqlite")
connection = sqlite3.connect(f"{sqlite_dir}2024031318__sqlite")
# burp2rdb

data_table = 'DATA'
header_table = 'HEADER'
# Define the SQL query you want to execute
query_data = f"SELECT * FROM {data_table};"
query_header = f"SELECT * FROM {header_table};"

# Execute the query and store the result in a DataFrame
result_data = pd.read_sql_query(query_data, connection)
result_data_arrow = pd.read_sql_query(query_data, connection, dtype_backend='pyarrow')
result_header = pd.read_sql_query(query_header, connection)

connection.close()

In [ ]:
result_data_pl_df = pl.from_pandas(result_data)

In [ ]:
print(type(result_data))
print(type(result_data_pl_df))

In [ ]:
delta_lake_df = pl.scan_delta("tar_swob_no_optimization")

In [ ]:
print(result_data.dtypes)

In [ ]:
print(result_data_pl_df.dtypes)

In [ ]:
# result_data_arrow = pd.read_sql_query(query_data, connection, dtype_backend='pyarrow')
print(result_data_arrow.dtypes)

In [ ]:
# result_data_arrow
# show(result_data_arrow, buttons=["copyHtml5", "csvHtml5", "excelHtml5"]) # ... (downsampled from 453,375x13 ...
show(result_data_pl_df.filter(pl.col("VARNO") == 12004)) # ... (downsampled from 25,065x13 to 856x13 as maxBytes=131072)

In [ ]:
result_data_pl_df

In [ ]:
# result_data_pl_df['VARNO'].unique()
varno_list = result_data_pl_df['VARNO'].unique().to_list()

# Generator expression to pad codes and join them with '|'
varno_list_grep = '|'.join(f"{code:06}" for code in varno_list)

#varno_list_grep = '|'.join(map(str, varno_df))

In [ ]:
varno_list_grep
# to supply to `grep -E` '^(001012|001013|002004|002038|005001|006001|007004|007030|007031|010004|010009|010051|010061|010062|010063|010194|011011|011012|011041|012002|012004|012006|012016|012017|013003|013013|013019|013020|013021|013022|013023|013033|013200|013204|020001|020003|020004|020005|020011|020012|020013|020062|020198|020199|020200|020201|020202|020213|022003|022011|022012|022013|022021|022022|022023|022042|022049)'

In [ ]:
code_list_df = pl.read_csv("codes_burp.csv", truncate_ragged_lines=True)

In [ ]:
code_list_df

In [ ]:
result_ID_OBS = pd.merge(result_data, result_header, on="ID_OBS")


In [ ]:
result_ID_OBS

In [ ]:
unique_time_values = result_ID_OBS["TIME"].unique()
print(unique_time_values)